<a href="https://colab.research.google.com/github/yosaamartyaAR/Tugas-Pembelajaran-Mesin/blob/main/Week7_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
import lightgbm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('/content/Telco_customer_churn.csv')
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn_rate,Churn Score,CLTV,Churn Reason,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Age,Under 30,Married,Referred a Friend,Number of Referrals,Offer,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Streaming Music,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Score.1,Churn Category
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,37,No,No,No,0,None,10.47,21,No,No,Yes,0.0,0,20.94,129.09,1,Churned,86,Competitor
1,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,19,Yes,No,No,0,None,9.12,51,No,No,Yes,0.0,0,18.24,169.89,2,Churned,67,Other
2,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,31,No,No,No,0,None,12.15,26,Yes,No,Yes,0.0,0,97.20,917.70,3,Churned,86,Other


In [3]:
# Let's delete data leakage columns and columns with 1 unique catigory
drop_cols = ['Churn Label', 'Churn Score', 'CLTV', 'Churn Reason', 'Count', 'Country', 'State', 'Lat Long', 'CustomerID']
df.drop(drop_cols, axis=1, inplace=True)

KeyError: ignored

In [ ]:
# We already know that NaNs are not mapped. But we have white spaces in data, so we will replace them with NaNs
# df.loc[df['Total Charges'] == ' ']
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# df.isna().sum()

In [ ]:
# Replace ',' with '.' to convert to float
to_be_numeric = ['Latitude', 'Longitude', 'Monthly Charges', 'Total Charges']

for col in to_be_numeric:
    df[col].replace(',', '.', regex=True, inplace=True)
    df[col] = pd.to_numeric(df[col])

In [ ]:
# Let's find catigorical columns
cat_columns = [cname for cname in df.columns if df[cname].dtype == "object"]

# We need 'categorical' type for categorical columns for lightgbm
for col in df.columns:
    if col in cat_columns:
        df[col] = df[col].astype('category')

In [ ]:
# Splitting the data 
X = df.drop('Churn Value', axis=1)
y = df['Churn Value']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Base model
lgb_base = lightgbm.LGBMClassifier(random_state=0)
lgb_base.fit(X_train, y_train, eval_set=(X_valid, y_valid), eval_metric='auc', verbose=0)

print(f'roc_auc of base model: {roc_auc_score(y_valid, lgb_base.predict(X_valid))}\n')
print(classification_report(y_valid, lgb_base.predict(X_valid)))
plot_confusion_matrix(lgb_base, X_valid, y_valid, values_format='d', display_labels=['Did not leave', 'Left'])

In [ ]:
%%time
param_grid = {
              'max_depth': [4, 9, -1],
              'num_leaves': [4, 9],
              'learning_rate': [0.1],
              'scale_pos_weight': [3],
              'n_estimators': [50, 100],
              'reg_lambda': [10, 15],
              'subsample': [0.9],
              'colsample_bytree': [0.5, 0.6]
              }

lgb = lightgbm.LGBMClassifier(random_state=1)

opt_params = GridSearchCV(estimator=lgb,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=3)

opt_params.fit(X_train, y_train, eval_metric='auc', eval_set=(X_valid, y_valid), verbose=0)
params = opt_params.best_params_
print(params)

In [ ]:
lgb = lightgbm.LGBMClassifier(**params)
lgb.fit(X_train, y_train)

print(f'roc_auc of final model: {roc_auc_score(y_valid, lgb.predict(X_valid))}\n')
print(classification_report(y_valid, lgb.predict(X_valid)))
plot_confusion_matrix(lgb, X_valid, y_valid, values_format='d', display_labels=['Did not leave', 'Left'])